In [2]:
import kagglehub
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    Dense, Dropout, BatchNormalization,
    GlobalAveragePooling2D
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
path = kagglehub.dataset_download(
    "lukechugh/best-alzheimer-mri-dataset-99-accuracy"
)

print("Dataset downloaded at:", path)
print("Root contents:", os.listdir(path))

Dataset downloaded at: /kaggle/input/best-alzheimer-mri-dataset-99-accuracy
Root contents: ['Combined Dataset']


In [4]:
BASE_DIR = os.path.join(path, "Combined Dataset")
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR  = os.path.join(BASE_DIR, "test")

print("Train classes:", os.listdir(TRAIN_DIR))
print("Test classes :", os.listdir(TEST_DIR))

Train classes: ['No Impairment', 'Very Mild Impairment', 'Moderate Impairment', 'Mild Impairment']
Test classes : ['No Impairment', 'Very Mild Impairment', 'Moderate Impairment', 'Mild Impairment']


In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True,
    zoom_range=0.3
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print("Class indices:", train_generator.class_indices)

Found 10240 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.
Class indices: {'Mild Impairment': 0, 'Moderate Impairment': 1, 'No Impairment': 2, 'Very Mild Impairment': 3}


In [7]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding="same", activation="relu",
                 input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(GlobalAveragePooling2D())

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4, activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1767775115.666663      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767775115.670500      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,172 (434.27 KB)

 Trainable params: 110,724 (432.52 KB)

 Non-trainable params: 448 (1.75 KB)

In [8]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1767775120.535884     135 service.cc:152] XLA service 0x7b7dc4107f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767775120.535922     135 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1767775120.535928     135 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1767775121.117781     135 cuda_dnn.cc:529] Loaded cuDNN version 91002


  2/320 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.1328 - loss: 1.6135

I0000 00:00:1767775127.285646     135 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


320/320 ━━━━━━━━━━━━━━━━━━━━ 136s 397ms/step - accuracy: 0.4874 - loss: 1.1416 - val_accuracy: 0.5004 - val_loss: 2.3789
Epoch 2/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 103s 321ms/step - accuracy: 0.6914 - loss: 0.6756 - val_accuracy: 0.4910 - val_loss: 1.0047
Epoch 3/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 104s 325ms/step - accuracy: 0.7288 - loss: 0.5983 - val_accuracy: 0.5403 - val_loss: 0.9447
Epoch 4/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 103s 323ms/step - accuracy: 0.7358 - loss: 0.5629 - val_accuracy: 0.5504 - val_loss: 0.9010
Epoch 5/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 104s 326ms/step - accuracy: 0.7442 - loss: 0.5358 - val_accuracy: 0.5035 - val_loss: 1.8406
Epoch 6/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 103s 322ms/step - accuracy: 0.7484 - loss: 0.5310 - val_accuracy: 0.5637 - val_loss: 0.9321
Epoch 7/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 107s 335ms/step - accuracy: 0.7598 - loss: 0.5197 - val_accuracy: 0.2525 - val_loss: 1.2902
Epoch 8/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 104s 323ms/step - accuracy: 0.7603 - loss: 0.50

In [9]:
test_loss, test_acc = model.evaluate(train_generator)
print("Test Accuracy:", test_acc)

320/320 ━━━━━━━━━━━━━━━━━━━━ 99s 310ms/step - accuracy: 0.7742 - loss: 0.4558
Test Accuracy: 0.77197265625


In [10]:
model.save("alzheimer_cnn_final.h5")

print("Model saved.")

Model saved.
